In [39]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import librosa
import numpy as np
import librosa.display
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from server.database_wrapper import PostgresqlWrapper
from server.utils import Util
from feature_extractor import FeatureExtractor
import xgboost as xgb
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Song downloading from database
db = PostgresqlWrapper(5)
data = db.fetch_songs(150, genres=['classical', 'metal', 'blues', 'hiphop', 'disco', 'pop', 'rock'])

In [65]:
data = list()
genre_list = list()

In [66]:
genres=['classical', 'metal', 'blues', 'hiphop', 'disco', 'pop', 'rock']

In [67]:
def getData(where_to, genre_list, genre, range_tuple):
    for i in tqdm(range(range_tuple)):
        if i < 10:
            path = "../../" + genre + "/" + genre + ".0000" + str(i) + ".au"        
        else:
            path = "../../" + genre + "/" + genre + ".000" + str(i) + ".au"
        
        song = librosa.load(path)
        where_to.append(song[0])
        genre_list.append(genre)

In [68]:
for genre in genres:
    getData(data, genre_list, genre, 50)

100%|██████████| 50/50 [00:27<00:00,  1.86it/s]


In [ ]:
# Create dataset
util = Util()
songs, genres = util.to_dataset(data)

In [80]:
# Extract features (mfcc means and zero_crossing_rate for now)
extractor = FeatureExtractor(data)
mfcc_means = extractor.generate_mfcc(n_mfcc=20, sr=22050)
zcrs = extractor.generate_zero_crossing_rate(None, None)
cent = extractor.generate_centoid_meanstd()

Got mfcc for 0 songs
Got mfcc for 50 songs
Got mfcc for 100 songs
Got mfcc for 150 songs
Got mfcc for 200 songs
Got mfcc for 250 songs
Got mfcc for 300 songs
Got zero_cross_rate for 0 songs
Got zero_cross_rate for 50 songs
Got zero_cross_rate for 100 songs
Got zero_cross_rate for 150 songs
Got zero_cross_rate for 200 songs
Got zero_cross_rate for 250 songs
Got zero_cross_rate for 300 songs
Got centroid data for 0 songs
Got centroid data for 50 songs
Got centroid data for 100 songs
Got centroid data for 150 songs
Got centroid data for 200 songs
Got centroid data for 250 songs
Got centroid data for 300 songs


In [81]:
X = np.hstack((mfcc_means, zcrs, cent))

In [82]:
X.shape

(350, 24)

In [83]:
y = LabelEncoder().fit_transform(genre_list)
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6,

In [111]:
# Prepare data for xgboost. Encode genres. Train/test split
y = LabelEncoder().fit_transform(genre_list)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [112]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [113]:
# Xgboost training
param = {'objective': "multi:softmax", "num_class": np.unique(y_train).size}
evallist = [(dtrain, 'train'), (dtest, 'eval')]
bst = xgb.train(param, dtrain, evals=evallist)

[0]	train-merror:0.102041	eval-merror:0.342857
[1]	train-merror:0.040816	eval-merror:0.295238
[2]	train-merror:0.016327	eval-merror:0.285714
[3]	train-merror:0.016327	eval-merror:0.247619
[4]	train-merror:0.008163	eval-merror:0.247619
[5]	train-merror:0	eval-merror:0.247619
[6]	train-merror:0	eval-merror:0.247619
[7]	train-merror:0	eval-merror:0.247619
[8]	train-merror:0	eval-merror:0.266667
[9]	train-merror:0	eval-merror:0.257143


In [114]:
print(classification_report(y_test, bst.predict(dtest), target_names=['classical', 'metal', 'blues', 'hiphop', 'disco', 'pop', 'rock']))

             precision    recall  f1-score   support

  classical       0.74      0.93      0.82        15
      metal       0.94      1.00      0.97        15
      blues       0.62      0.67      0.65        15
     hiphop       0.89      0.53      0.67        15
      disco       0.80      0.80      0.80        15
        pop       0.85      0.73      0.79        15
       rock       0.47      0.53      0.50        15

avg / total       0.76      0.74      0.74       105



In [100]:
# Pay attention to rock. SICK!